In [1]:
import pandas as pd
df_letters = pd.read_csv('all_lettters_with_entities_final.tsv',sep='\t')
df_manual = pd.read_excel('Network_from_Carlotta/Correspondence_analysis.xlsx',sep='\t').dropna(subset=['Sender (From)','Addressee (To)'])
filenames_manual = set(df_manual['File name'].tolist())
df_graph = pd.read_csv('df_graph_with_wikidata.tsv',sep='\t')
df_letters['is_seen_in_manual'] = df_letters['filename'].map(lambda x: x in filenames_manual)
df_letters_interesting = df_letters[df_letters['is_seen_in_manual']]

In [2]:
import json
aka = json.load(open('also_known_as_final.json','r'))

alias = {}
for main_name, names in aka.items():
    for name in names:
        alias[name] = main_name
        
def resolve_alias(p):
    cnt = 0
    while p in alias and cnt < 10:
        p = alias[p]
        cnt += 1
    if cnt == 10:
        print('Got a loop in aliases! Starting point: ',p)
    return p


In [3]:
people_with_wikidata = set(df_graph['person1'].tolist()+df_graph['person2'].tolist())
df_letters_interesting['people'] = df_letters_interesting['people_split'].map(lambda x: set([resolve_alias(person) for person in eval(x)
                                                            if resolve_alias(person) in people_with_wikidata]))
df_letters_interesting['has_people'] = df_letters_interesting['people'].map(lambda x: len(x)>1)
df_letters_interesting = df_letters_interesting[df_letters_interesting['has_people']].reset_index()
df_letters_interesting['filename'] = df_letters_interesting['filename'].astype(int)
df_letters_interesting.head()

/home/vprovat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/vprovat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,index,part_of.title,images.reference,filename,is_missing,path_to_image,page_type,path_to_text,path_to_handwritten_info,people,handwritten_info,people_split,is_seen_in_manual,has_people
0,744,Correspondence with the Victoria & Albert Muse...,..\images\jpg\922\0001248922.jpg,1248922,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,"{Harvey Littleton, Wilke Adolfsson, Durk Valke...",NaN,"{'Victoria and', 'J.V.G.Mallet', 'Durk', 'arff...",True,True
1,746,Correspondence with the Victoria & Albert Muse...,..\images\jpg\942\0001248942.jpg,1248942,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,"{Yvonne Brunhammer, Wilke Adolfsson}",NaN,"{'Wilke', 'G.Maliet', 'Wilke adolfsson', 'Yvon...",True,True
2,756,"Correspondence with Jon Andersson, West Surrey...",..\images\jpg\960\0001248960.jpg,1248960,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,"{Mieke Groot, Richard Meitner, John Anderson, ...",NaN,"{'Engle d Bla', 'Jon Sounds', 'Fred Roeske', '...",True,True
3,757,"Correspondence with Jon Andersson, West Surrey...",..\images\jpg\964\0001248964.jpg,1248964,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,"{John Anderson, Gerrit Rietveld, Sybren Valkema}",NaN,"{'Gerrit Rietveld', 'Syben Valkema', 'Jon Ande...",True,True
4,826,Correspondence with 'Instituut voor Industriel...,..\images\jpg\393\0001258393.jpg,1258393,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,"{Mart Stam, René Smeets, Donald Holden, Sybren...",NaN,"{'W. R', 'Arch. G. Rietveld', 'R. Smeets', 'Cr...",True,True


In [6]:
graph = {}

In [4]:
def add_edge(p1, p2, doc_id):
    if p1 == p2:
        return
    
    if (p1, p2) in graph:
        graph[(p1,p2)].append(doc_id)
    else:
        graph[(p1,p2)] = [doc_id]
        
        
def add_edges(people, doc_id):
    '''
    people: list of people found in a document
    doc_id: path to text or another unique identifier
    '''
    if len(people) < 2:
        return
    
    for i, p1_raw in enumerate(people):
        for p2_raw in people[i:]:
            p1 = resolve_alias(p1_raw)
            p2 = resolve_alias(p2_raw)
            add_edge(p1, p2, doc_id)
            add_edge(p2, p1, doc_id)

In [5]:
def stops_here(cur_df):
    if cur_df.empty:
        return True
    try:
        text = open(cur_df.iloc[0]['path_to_text'],'r').read().lower()
    except:
        print('Failed to find text for: ',cur_df.iloc[0]['path_to_text'])
        text = ''
    if text.find('geachte') != -1:
        return True
    if text.find('geehrter') != -1:
        return True
    if text.find('dear') != -1:
        return True
    return False

In [8]:
def get_next_file(filename):
    name = filename+1
    return df_letters_interesting[df_letters_interesting['filename']==name]

In [9]:
from tqdm.auto import tqdm

processed_filenames = set()
for i in tqdm(range(0, len(df_letters_interesting))):
    row = df_letters_interesting.iloc[i]
    if row['filename'] in processed_filenames: # it was someone else's second page already
        continue 
    processed_filenames.add(row['filename'])

    cur_people = set(eval(row['people'])) if isinstance(row['people'],str) else row['people']

    cur_id = row['filename']
    
    df_next = get_next_file(row['filename'])
    while not stops_here(df_next):
        next_row = df_next.iloc[0]
        processed_filenames.add(next_row['filename'])

        next_people = set(eval(next_row['people'])) if isinstance(next_row['people'],str) else next_row['people']
        cur_people = cur_people.union(next_people)

        cur_id += '-'+ extract_id(next_row['path_to_image'])
        
        df_next = get_next_file(next_row['filename'])
        
    add_edges(list(cur_people), cur_id)

  0%|          | 0/132 [00:00<?, ?it/s]

In [11]:
wikidata_id_per_name = dict(zip(df_graph['person1'].tolist()+df_graph['person2'].tolist(), 
                                df_graph['wikidata_id1'].tolist()+df_graph['wikidata_id2'].tolist()
                               ))

wikidata_description_per_name = dict(zip(df_graph['person1'].tolist()+df_graph['person2'].tolist(), 
                                df_graph['wikidata_description1'].tolist()+df_graph['wikidata_description2'].tolist()
                               ))

main_country_per_name = dict(zip(df_graph['person1'].tolist()+df_graph['person2'].tolist(), 
                                df_graph['main_country1'].tolist()+df_graph['main_country2'].tolist()
                               ))

In [12]:
# saving to a readable format

records = []
seen_connections = set()
for people, docs in graph.items():
    person1 = min(people)
    person2 = max(people)
    if (person1, person2) in seen_connections:
        continue
    seen_connections.add((person1,person2))
    records.append({'person1':person1,
                   'person2':person2,
                   'document_ids':docs,
                   'weight':len(docs),
                   'wikidata_id1':wikidata_id_per_name[person1],
                   'wikidata_id2':wikidata_id_per_name[person2],
                   'wikidata_description1':wikidata_description_per_name[person1],
                   'wikidata_description2':wikidata_description_per_name[person2],
                   'main_country1': main_country_per_name[person1],
                   'main_country2': main_country_per_name[person2]})
    
df_graph_small = pd.DataFrame.from_records(records)
df_graph_small

,person1,person2,document_ids,weight,wikidata_id1,wikidata_id2,wikidata_description1,wikidata_description2,main_country1,main_country2
0,Harvey Littleton,Wilke Adolfsson,[1248922],1,Q15488059,Q5543008,"Artist, educator",Swedish artist,United States of America,Sweden
1,Durk Valkema,Harvey Littleton,"[1248922, 1265626]",2,Q29422094,Q15488059,Dutch sculptor,"Artist, educator",Kingdom of the Netherlands,United States of America
2,Harvey Littleton,Sybren Valkema,"[1248922, 1266713, 1268564, 1268576, 1268586, ...",11,Q15488059,Q2618110,"Artist, educator",Dutch glass artist (1916-1996),United States of America,Kingdom of the Netherlands
3,Gary Beecham,Harvey Littleton,"[1248922, 1290868]",2,Q5524683,Q15488059,American artist,"Artist, educator",United States of America,United States of America
4,Durk Valkema,Wilke Adolfsson,"[1248922, 1468401, 1266688]",3,Q29422094,Q5543008,Dutch sculptor,Swedish artist,Kingdom of the Netherlands,Sweden
...,...,...,...,...,...,...,...,...,...,...
439,Helmut Ricke,Kea Verwey,[1461125],1,Q95328683,Q29436115,German art historian and museum director,Dutch sculptor,NaN,Kingdom of the Netherlands
440,Richard Meitner,Roberto Niederer,[1461188],1,Q22236132,Q78067540,Description not found.,"Swiss entrepreneur, glassblower and artist. He...",United States of America,Switzerland
441,Floris Meydam,Roberto Niederer,[1461188],1,Q2312612,Q78067540,Description not found.,"Swiss entrepreneur, glassblower and artist. He...",Kingdom of the Netherlands,Switzerland
442,Mieke Groot,Roberto Niederer,[1461188],1,Q96756232,Q78067540,Dutch glass artist,"Swiss entrepreneur, glassblower and artist. He...",NaN,Switzerland


In [13]:
df_graph_small.to_csv('df_graph_same_docs_as_manual.tsv',sep='\t',index=False)